# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier


In [4]:
from sqlalchemy import create_engine
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')



# create the test table including project_id as a primary key
df = pd.read_sql_table('DisasterResponse', 'sqlite:///DisasterResponse.db')
df.head()





,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
a = df.loc[:,'related':'direct_report']
a.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df['message']
Y = df.loc[:,'related':'direct_report']

In [7]:
Y.shape

(26216, 36)

In [8]:
X.shape

(26216,)

### 2. Write a tokenization function to process your text data

In [9]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    clean_tokens = [word for word in clean_tokens if word not in stopwords.words('english')]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [12]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
Y_pred = pipeline.predict(X_test)

In [14]:
Y_test.iloc[:,0]

9264     1
14621    1
11732    0
14275    1
23690    1
16532    0
4041     1
7432     1
8309     0
13499    1
6182     1
21198    0
20719    1
23951    1
16287    1
16449    0
5738     0
17389    1
4181     1
25832    0
25303    1
25218    1
6502     0
21589    1
13478    1
16810    0
2422     1
9555     1
2245     1
9331     0
        ..
17879    1
4223     0
13178    1
21483    0
21208    1
8412     0
20050    1
926      1
17316    1
22710    1
13494    1
17752    0
18357    1
11224    1
12469    1
23699    0
15571    0
23122    0
5836     1
11293    1
13345    1
7550     1
6076     1
1116     1
18155    1
8792     0
3116     0
18301    1
13007    1
5874     0
Name: related, Length: 6554, dtype: int64

In [15]:
Y_pred[:,0]

array([1, 1, 1, ..., 1, 1, 0])

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score
classification_report = []
for i in range(len(Y_test.columns)):
    classification_report.append((
        Y_test.columns[i],
        precision_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro'),
        recall_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro'),
        f1_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro')       
    ))

In [17]:
cr = pd.DataFrame(np.array(classification_report).reshape(36,4))
cr.columns = ['class name', 'precision score','recall score','f1 score']
print(cr)

                class name precision score    recall score        f1 score
0                  related  0.801953005798  0.801953005798  0.801953005798
1                  request  0.881141287763  0.881141287763  0.881141287763
2                    offer  0.994964906927  0.994964906927  0.994964906927
3              aid_related  0.739090631675  0.739090631675  0.739090631675
4             medical_help  0.918675617943  0.918675617943  0.918675617943
5         medical_products   0.95346353372   0.95346353372   0.95346353372
6        search_and_rescue  0.972688434544  0.972688434544  0.972688434544
7                 security  0.983521513579  0.983521513579  0.983521513579
8                 military  0.966432712847  0.966432712847  0.966432712847
9              child_alone             1.0             1.0             1.0
10                   water  0.944613976198  0.944613976198  0.944613976198
11                    food  0.932560268538  0.932560268538  0.932560268538
12                 shelte

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [19]:
from sklearn.model_selection import GridSearchCV
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [1, 2, 3],
        
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:

cv.fit(X_train, Y_train)


In [1]:
Y_pred = cv.predict(X_test)

NameError: name 'cv' is not defined

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
from sklearn.metrics import accuracy_score
classification_report = []
for i in range(len(Y_test.columns)):
    classification_report.append((
        Y_test.columns[i],
        precision_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro'),
        recall_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro'),
        accuracy_score(Y_test.iloc[:,i],Y_pred[:,i])       
    ))

In [22]:
cr = pd.DataFrame(np.array(classification_report).reshape(36,4))
cr.columns = ['class name', 'precision score','recall score','accuracy_score']
print(cr)

                class name precision score    recall score  accuracy_score
0                  related  0.614621489621  0.614621489621  0.614621489621
1                  request  0.816239316239  0.816239316239  0.816239316239
2                    offer  0.995115995116  0.995115995116  0.995115995116
3              aid_related  0.570818070818  0.570818070818  0.570818070818
4             medical_help  0.911324786325  0.911324786325  0.911324786325
5         medical_products  0.945207570208  0.945207570208  0.945207570208
6        search_and_rescue  0.971611721612  0.971611721612  0.971611721612
7                 security  0.983821733822  0.983821733822  0.983821733822
8                 military  0.968864468864  0.968864468864  0.968864468864
9              child_alone             1.0             1.0             1.0
10                   water  0.935592185592  0.935592185592  0.935592185592
11                    food  0.881715506716  0.881715506716  0.881715506716
12                 shelte

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(sentence)

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

In [14]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

In [15]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [35]:
2+2

4

In [16]:
Y_pred = pipeline.predict(X_test)

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score
classification_report = []
for i in range(len(Y_test.columns)):
    classification_report.append((
        Y_test.columns[i],
        precision_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro'),
        recall_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro'),
        f1_score(Y_test.iloc[:,i],Y_pred[:,i],average='micro')       
    ))

In [34]:
cr = pd.DataFrame(np.array(classification_report).reshape(36,4))
cr.columns = ['class name', 'precision score','recall score','accuracy_score']
print(cr)

                class name precision score    recall score  accuracy_score
0                  related  0.680097680098  0.680097680098  0.680097680098
1                  request  0.820207570208  0.820207570208  0.820207570208
2                    offer  0.995421245421  0.995421245421  0.995421245421
3              aid_related  0.578601953602  0.578601953602  0.578601953602
4             medical_help  0.917277167277  0.917277167277  0.917277167277
5         medical_products  0.948717948718  0.948717948718  0.948717948718
6        search_and_rescue   0.97268009768   0.97268009768   0.97268009768
7                 security  0.984126984127  0.984126984127  0.984126984127
8                 military  0.970543345543  0.970543345543  0.970543345543
9              child_alone             1.0             1.0             1.0
10                   water   0.93894993895   0.93894993895   0.93894993895
11                    food  0.886752136752  0.886752136752  0.886752136752
12                 shelte

### 9. Export your model as a pickle file

In [17]:
from sklearn.externals import joblib
filename = 'model.pkl'
joblib.dump(pipeline, filename)

['model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.